# Parallel Inductance Design

Main eqs from Annunziata paper:

(1)  $L_K= \frac {l}{w} \frac{R_{sq}h}{2\pi^2 \Delta} \frac{1}{\tanh{\frac{\Delta}{2k_BT}}}$  

(2) $\Delta = \Delta_{00} e^{\frac{-\pi\xi}{4}}$

(3) $\frac{I}{I_c}=1.897 e^{\frac{-3\pi\xi}{8}}\sqrt{\xi}(\frac{\pi}{2}-\frac{2}{3}\xi)$

(5) $f=\frac{1}{\sqrt{(L_m+L_k)C}}$



In [27]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
import math

#Problem Parameters
m = 9.1093e-31  ## electron mass
e = 1.60217e-17 ## electron charge
l=100e-6 ## length of the nanowire                 
Jc=6e10 ## Critical current density from literature (A/m^2)
Tc=11 ## Critical Temperature From literature
T=2e-3 ##operating temperature
Kb=1.38e-23 ##Boltzman Constant
del00= 1.76*Kb*Tc ##Energy Gap at zero temp and zero bias current
ro=200e-8 ## resistivity of NbTiN from literature   ##not accurate
h=6.626e-34 ##Plank's Constant
fin=int(input('Enter the operating freq in GHz ')) #Input Frequency
fin=fin*1e9


#Our Main variables
I=int(input('Enter the bias current value in uA ')) ##Bias Current 
I=I*1e-6
Lm=300e-12 ##Geometric Inductance (Assumption)
C=10e-12 ##Resonator Capacitance (Assumption)


def eq3(xi):
    return [I/Ic-(1.897*(math.exp(-3*np.pi*xi[0]/8))*math.sqrt(xi[0])*(np.pi/2-2*xi[0]/3))] ##equation3
     
def eq5(f):     
    return [f[0]-(1/(math.sqrt((Lm+Lk)*C)))] ##equation5

## Inductor#1
w1=100e-9 ## width of the nanowire                   
t1=10e-9 ## Thickness of the nanowire  
Ic=w1*t1*Jc ## Critical current 
print ("\nCritical current for 1st inductor is", Ic*1e6, "uA")
if I<Ic:
    xi = fsolve(eq3,[0]) ##eq3
    delta=del00*math.exp(-np.pi*xi/4)##eq2
    Lk1=(l/(w1*t1))*(ro*h)/(2*(np.pi**2)*delta*math.tanh(delta/(2*Kb*T))) ##equation1    
    print ("Lk1 is equal to", Lk1*1e9, "nH")   
else:
    Lk1=0
    print ("no contribution")

## Inductor#2
w2=200e-9 ## width of the nanowire                   
t2=20e-9 ## Thickness of the nanowire  
Ic=w2*t2*Jc ## Critical current 
print ("\nCritical current for 2nd inductor is", Ic*1e6, "uA")
if I<Ic :
    xi = fsolve(eq3,[0]) ##eq3
    delta=del00*math.exp(-np.pi*xi/4)##eq2
    Lk2=(l/(w2*t2))*(ro*h)/(2*(np.pi**2)*delta*math.tanh(delta/(2*Kb*T))) ##equation1    
    print ("Lk2 is equal to", Lk2*1e9, "nH")
else:    
    Lk2=0
    print ("no contribution")

## Inductor#3
w3=300e-9 ## width of the nanowire                   
t3=30e-9 ## Thickness of the nanowire  
Ic=w3*t3*Jc## Critical current 
print ("\nCritical current for 3rd inductor is", Ic*1e6, "uA")
if I<Ic :
    xi = fsolve(eq3,[0]) ##eq3
    delta=del00*math.exp(-np.pi*xi/4)##eq2
    Lk3=(l/(w3*t3))*(ro*h)/(2*(np.pi**2)*delta*math.tanh(delta/(2*Kb*T))) ##equation1    
    print ("Lk3 is equal to", Lk3*1e9, "nH")
else:   
    Lk3=0     
    print ("no contribution")
    

#calculation of Lk0
I=0

xi = fsolve(eq3,[0]) ##eq3
delta=del00*math.exp(-np.pi*xi/4)##eq2
Lk01=(l/(w1*t1))*(ro*h)/(2*(np.pi**2)*delta*math.tanh(delta/(2*Kb*T))) ##equation1  

xi = fsolve(eq3,[0]) ##eq3
delta=del00*math.exp(-np.pi*xi/4)##eq2
Lk02=(l/(w2*t2))*(ro*h)/(2*(np.pi**2)*delta*math.tanh(delta/(2*Kb*T))) ##equation1    

xi = fsolve(eq3,[0]) ##eq3
delta=del00*math.exp(-np.pi*xi/4)##eq2
Lk03=(l/(w3*t3))*(ro*h)/(2*(np.pi**2)*delta*math.tanh(delta/(2*Kb*T))) ##equation1    

Lk0= (((Lk01)**-1)+((Lk02)**-1)+((Lk03)**-1))**-1
print ("\nLk0 is equal to", Lk0*1e9, "nH")


#Overall Inductance Lk and Impedence
if Lk1==0:
    R_L1=(ro*l)/(w1*t1) #Resistance
    if Lk2==0:
        R_L2=(ro*l)/(w2*t2)  #Resistance
        if Lk3==0:
            R_L3=(ro*l)/(w3*t3) #Resistance
            R_L=(((ro*l)/(w3*t3))**-1+((ro*l)/(w2*t2))**-1+((ro*l)/(w1*t1))**-1)**-1
            Lk=0
        else:
            Lk=((Lk3)**-1)**-1
            R_L3=0 #Resistance 
            R_L=(((ro*l)/(w2*t2))**-1+((ro*l)/(w1*t1))**-1)**-1
    else: 
        Lk=(((Lk2)**-1)+((Lk3)**-1))**-1
        R_L2=0 #Resistance 
        R_L=(((ro*l)/(w1*t1))**-1)**-1
else:            
    Lk=(((Lk1)**-1)+((Lk2)**-1)+((Lk3)**-1))**-1
    R_L1=0 #Resistance 
    R_L=0
    
    
#Overall Impedence    
if Lk1==0:
    X_L1=2*np.pi*fin*Lk01 #Impedence
else:            
    X_L1=2*np.pi*fin*Lk1  #Impedence
if Lk2==0:
    X_L2=2*np.pi*fin*Lk02  #Impedence
else: 
    X_L2=2*np.pi*fin*Lk2  #Impedence

if Lk3==0:
    X_L3=2*np.pi*fin*Lk03  #Impedence
else:
    X_L3=2*np.pi*fin*Lk3  #Impedence


print ("\nLk is equal to", Lk*1e9, "nH")


#Change in Frequency
f0=1/(math.sqrt((Lm+Lk0)*C))
print('\nfrequency f0 is equal to', f0*1e-9, 'GHz')
fnew=1/(math.sqrt((Lm+Lk)*C))
print('New frequency is equal to', fnew*1e-9, 'GHz')
delf= np.abs(fnew-f0)
print('Change in frequency is equal to', delf*1e-6, 'MHz')


#Calculating the overall impedence impedence
X_L= (((X_L1)**-1)+((X_L2)**-1)+((X_L3)**-1))**-1 #Overall Impedence
print('\nOverall Impedence is equal to', X_L, ' ohm')
print('Overall Resistence is equal to', R_L, ' ohm')



Enter the operating freq in GHz 9
Enter the bias current value in uA 500

Critical current for 1st inductor is 59.99999999999999 uA
no contribution

Critical current for 2nd inductor is 239.99999999999997 uA
no contribution

Critical current for 3rd inductor is 540.0 uA
Lk3 is equal to 3.1767284524564685 nH

Lk0 is equal to 1.7948956750972371 nH

Lk is equal to 3.1767284524564685 nH

frequency f0 is equal to 6.909057386151253 GHz
New frequency is equal to 5.363084184315256 GHz
Change in frequency is equal to 1545.9732018359966 MHz

Overall Impedence is equal to 110.06692776932958  ohm
Overall Resistence is equal to 40000.0  ohm
